# **Deep Learning For Lung Cancer Prediction Based on Transcriptomic Data : a Basic Convolutional Neural Network with Transfer Learning**
> Author : **Aymen MERROUCHE**. <br>
> In this notebook, we implement a basic CNN for our binary classification task. First we pre train our CNN on the non Lung cancer dataset. Then, in a transfer learning approach, we fine tune it on the lung cancer dataset (we don't freeze the convolutional layers, and we don't keep the dense layers meaninig that only the convolutional layers are kept) :

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import datetime

from utils import *
from train import *
from data_utils import *
from modules.basic_cnn import *
from modules.focal_loss import *
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings
warnings.filterwarnings('always')

In [2]:
# device to use, if cuda available then use cuda else use cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Working on : ", device)

Working on :  cuda


In [3]:
# load hyperparametrs
# data paths args
with open('./configs/data_paths.yaml', 'r') as stream:
    data_paths_args  = yaml.load(stream,Loader=yaml.Loader)
    
    
# basic cnn args
with open('./configs/basic_cnn.yaml', 'r') as stream:
    basic_cnn_args  = yaml.load(stream,Loader=yaml.Loader)

## **1 - Pre-Training on the Non Lung Dataset :**

### **1 - 1 - Get the Data :**

In [4]:
%%time
# Getting the data
# dataset
non_lung_dataset = TranscriptomicImagesDatasetNonLung(data_paths_args["path_to_pan_cancer_hdf5_files"], data_paths_args["path_to_treemap_images"])
non_lung_dataloader_train, non_lung_dataloader_validation = get_data_loaders(non_lung_dataset, batch_size_train = basic_cnn_args["batch_size_pt"],\
                                                                             batch_size_validation = basic_cnn_args["batch_size_pt"])

CPU times: user 4.82 s, sys: 1.04 s, total: 5.86 s
Wall time: 3.11 s


### **1 - 2 - Network, Criterion and Training :**

In [5]:
# network
net = Basic_CNN().to(device).double()

# loss and optimizer  
criterion = FocalLoss().to(device)
optimizer = optim.Adam(net.parameters(), lr=basic_cnn_args['lr_pt'])

# Logging + Experiment

ignore_keys = {'no_tensorboard'}
# get hyperparameters with values in a dict
hparams = {**basic_cnn_args}
# generate a name for the experiment
expe_name = '_'.join([f"{key}={val}" for key, val in hparams.items()])
print("Experimenting with : \n \t"+expe_name)
# path where to save the model
savepath = Path('models/pre_trained_cnn_checkpt.pt')
# Tensorboard summary writer
if basic_cnn_args['no_tensorboard']:
    writer = None
else:
    writer = SummaryWriter("runs/runs"+"_"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+expe_name)
    
# start the experiment
checkpoint = CheckpointState(net, optimizer, savepath=savepath)
fit(checkpoint, criterion, non_lung_dataloader_train, non_lung_dataloader_validation, basic_cnn_args['epochs'], writer=writer)
if not basic_cnn_args['no_tensorboard']:
    writer.close()

Epoch 1/50:   0%|          | 0/41 [00:00<?, ?it/s, loss=2.3246e-01]

Experimenting with : 
 	epochs=50_batch_size_pt=128_lr_pt=0.001_batch_size_ft=128_lr_ft=0.001_no_tensorboard=True
Training on GPU 



Epoch 1/50: 100%|██████████| 41/41 [00:12<00:00,  3.23it/s, loss=1.5731e-01]
/users/Etu3/3802993/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/users/Etu3/3802993/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/users/Etu3/3802993/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavio

Epoch 1/50, Train Loss: 1.9766e-01, Test Loss: 0.0717
Epoch 1/50, Train Accuracy: 50.30%, Test Accuracy: 88.36%
Epoch 1/50, Train AUC: 53.44%, Test AUC: 52.26%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.88      1.00      0.94      2248
      Cancer       0.00      0.00      0.00       295

    accuracy                           0.88      2543
   macro avg       0.44      0.50      0.47      2543
weighted avg       0.78      0.88      0.83      2543



Epoch 2/50: 100%|██████████| 41/41 [00:12<00:00,  3.25it/s, loss=1.7521e-01]
/users/Etu3/3802993/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/users/Etu3/3802993/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/users/Etu3/3802993/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavio

...

Epoch 21/50, Train Loss: 4.6737e-02, Test Loss: 0.1529
Epoch 21/50, Train Accuracy: 81.48%, Test Accuracy: 61.83%
Epoch 21/50, Train AUC: 92.90%, Test AUC: 74.73%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.94      0.60      0.74      2248
      Cancer       0.19      0.72      0.30       295

    accuracy                           0.62      2543
   macro avg       0.57      0.66      0.52      2543
weighted avg       0.86      0.62      0.69      2543



Epoch 22/50: 100%|██████████| 41/41 [00:12<00:00,  3.16it/s, loss=1.2011e-01]


Epoch 22/50, Train Loss: 4.4078e-02, Test Loss: 0.1544
Epoch 22/50, Train Accuracy: 82.15%, Test Accuracy: 60.99%
Epoch 22/50, Train AUC: 94.13%, Test AUC: 75.09%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.95      0.59      0.73      2248
      Cancer       0.20      0.77      0.31       295

    accuracy                           0.61      2543
   macro avg       0.57      0.68      0.52      2543
weighted avg       0.86      0.61      0.68      2543



Epoch 23/50: 100%|██████████| 41/41 [00:13<00:00,  3.15it/s, loss=1.6526e-02]


KeyboardInterrupt: 

## **2 - Fine-Tuning on the Lung dataset :**

### **2 - 1 - Load Pre-Trained Model :**

In [6]:
# Load the pretrained Model
net = Basic_CNN().to(device).double()
optimizer = optim.Adam(net.parameters(), lr=basic_cnn_args['lr_ft'])
# path where the pre-trained model is saved : defined above+"_best"
savepath = Path('models/pre_trained_cnn_checkpt_best.pt')
checkpoint = CheckpointState(net, optimizer, savepath=savepath)
checkpoint.load()
pretrained = checkpoint.model

### **2 - 2 - Get the Data :**

In [7]:
%%time
# Getting the data
# dataset
lung_dataset = TranscriptomicImagesDatasetLung(data_paths_args["path_to_pan_cancer_hdf5_files"], data_paths_args["path_to_treemap_images"])
lung_dataloader_train, lung_dataloader_validation = get_data_loaders(lung_dataset, batch_size_train = basic_cnn_args["batch_size_ft"],\
                                                                             batch_size_validation = basic_cnn_args["batch_size_ft"])

CPU times: user 1.47 s, sys: 542 ms, total: 2.01 s
Wall time: 10.1 s


### **2 - 3 - Fine Tuning Procedure :**

In [8]:
# Beginnig Of Transfer Learnig Procedure
net = fine_tune_CNN(pretrained)
net = net.to(device).double()
criterion = FocalLoss().to(device)
optimizer = optim.Adam(net.parameters(), lr=basic_cnn_args['lr_ft'])
savepath = Path('models_finetuned/fine_tuned_cnn_checkpt.pt')
checkpoint = CheckpointState(net, optimizer, savepath=savepath)
fit(checkpoint, criterion, lung_dataloader_train, lung_dataloader_validation, basic_cnn_args['epochs'])

Epoch 1/50:   0%|          | 0/5 [00:00<?, ?it/s, loss=2.5017e-01]

Training on GPU 



Epoch 1/50: 100%|██████████| 5/5 [00:01<00:00,  3.46it/s, loss=1.6994e-01]


Epoch 1/50, Train Loss: 7.9098e-02, Test Loss: 0.0452
Epoch 1/50, Train Accuracy: 73.97%, Test Accuracy: 82.37%
Epoch 1/50, Train AUC: 85.30%, Test AUC: 66.14%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.93      0.86      0.90       260
      Cancer       0.14      0.27      0.19        22

    accuracy                           0.82       282
   macro avg       0.54      0.57      0.54       282
weighted avg       0.87      0.82      0.84       282



Epoch 2/50: 100%|██████████| 5/5 [00:01<00:00,  3.57it/s, loss=6.1928e-02]


...

Epoch 26/50: 100%|██████████| 5/5 [00:01<00:00,  3.57it/s, loss=2.4318e-03]


Epoch 26/50, Train Loss: 2.0057e-02, Test Loss: 0.3449
Epoch 26/50, Train Accuracy: 84.44%, Test Accuracy: 51.82%
Epoch 26/50, Train AUC: 99.90%, Test AUC: 66.42%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.96      0.50      0.66       260
      Cancer       0.12      0.77      0.20        22

    accuracy                           0.52       282
   macro avg       0.54      0.64      0.43       282
weighted avg       0.90      0.52      0.63       282



Epoch 27/50: 100%|██████████| 5/5 [00:01<00:00,  3.58it/s, loss=1.9782e-03]


Epoch 27/50, Train Loss: 2.2799e-02, Test Loss: 0.3623
Epoch 27/50, Train Accuracy: 84.89%, Test Accuracy: 51.04%
Epoch 27/50, Train AUC: 99.86%, Test AUC: 66.19%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.96      0.49      0.65       260
      Cancer       0.11      0.77      0.20        22

    accuracy                           0.51       282
   macro avg       0.54      0.63      0.43       282
weighted avg       0.90      0.51      0.62       282



KeyboardInterrupt: 